In [ ]:
import pandas as pd
import os
import numpy as np

# 绘图
import matplotlib.pyplot as plt


# 基本工具
import matplotlib
import platform
import numpy as np
import os
import pandas as pd


# 样本数据处理
from sklearn.model_selection import train_test_split

# 分类模型
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import plot_roc_curve

# 回归模型
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_percentage_error, r2_score, mean_squared_error, mean_absolute_error, mean_squared_log_error

# 保存模型
import joblib

In [ ]:
sys_platform = platform.platform().lower()
if "windows" in sys_platform:
    font = {
        "family": "Times New Roman"
    }
    matplotlib.rc("font", **font)
else:
    font = {
        "family": "Arial Unicode MS"
    }
    matplotlib.rc("font", **font)
rc = {"mathtext.fontset": "stix", }

plt.rcParams.update(rc)

In [ ]:
## 图像显示中文的问题，需要判断系统是windows还是苹果的

sys_platform = platform.platform().lower()
if "windows" in sys_platform:
    font = {
        "family": "Microsoft YaHei"
    }
    matplotlib.rc("font", **font)
else:
    font = {
        "family": "Arial Unicode MS"
    }
    matplotlib.rc("font", **font)

In [ ]:
def combine_csv(path):
    """
    将多个设备计算出的csv文件进行合并
    :param path: cs文件的存放路径
    :return: 返回合并后的DataFrame格式数据
    """
    files = os.listdir(path)
    design_res = pd.read_csv(path + "/" + files[0], )

    for file in files[1:]:
        df = pd.read_csv(path + "/" + file)
        design_res = pd.concat([design_res, df], axis=0)
    design_res = design_res.drop_duplicates()  # 去重
    design_res = design_res.reset_index(drop=True)  # 重新生成index
    design_res = design_res.loc[:, ~design_res.columns.str.contains('Unnamed')]
    return design_res

In [ ]:
data_dir = "./data"
if os.path.exists(data_dir):
    shutil.rmtree(data_dir)
os.mkdir(data_dir, 0o777)
design_res = combine_csv("./design_res_final_beta")
tar_file = os.path.join(data_dir, "design_res_beta.csv")
# 存储为csv文件
design_res.to_csv(tar_file, index=False)

In [ ]:
df=pd.read_csv(tar_file)
df.info()
df.sample(3)

In [ ]:
df = df.dropna(axis=0, how='any')

# 分类问题

In [ ]:
# 去掉id,site_name,mu,zeta,kappa,gamma,alpha,
x_label = ["period", "site_no", "wg", "xig", "stru_damping", "gamma_t"]
y_label = ["ok"]
X = df.loc[:, x_label].values
y = df.loc[:, y_label].values
y = np.array(y, dtype=bool)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=2)
y_test = y_test.reshape([-1,])
y_train = y_train.reshape([-1,])

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
new_X_train = X_train
new_X_test = X_test

## 随机森林

In [ ]:
# 贝叶斯优化
rfc = RandomForestClassifier()

param_grid = {'max_depth': np.arange(start=10, stop=500, step=10, dtype=int),
              'n_estimators': np.arange(start=50, stop=1000, step=50, dtype=int),
              'max_features': np.arange(start=1, stop=12, step=1, dtype=int),
              'min_samples_leaf': np.array([2, 5, 10, 15], dtype=int),
              'min_samples_split': np.array([2, 5, 10, 15], dtype=int),
              'random_state': [3]}

opt = BayesSearchCV(rfc, param_grid, n_iter=50, n_jobs=5, cv=3)
opt.fit(new_X_train, y_train)

In [ ]:
print(opt.best_params_)
best_params = opt.best_params_

In [ ]:

df["ok"].value_counts()

In [ ]:
rfc = RandomForestClassifier(n_estimators=best_params["n_estimators"], max_features=best_params['max_features'],
                             max_depth=best_params["max_depth"], min_samples_split=best_params["min_samples_split"],
                             min_samples_leaf=best_params["min_samples_leaf"], random_state=best_params["random_state"])
rfc.fit(new_X_train, y_train)

# 预测时间
start = time.time()
print("开始预测！！")
y_pre_rfc = rfc.predict(new_X_test)
end = time.time()
print('预测时间', end - start, 's')


print('正确标签：', y_test)
print('预测结果：', y_pre_rfc)

print('训练集分数：', rfc.score(new_X_train, y_train))
print('测试集分数：', rfc.score(new_X_test, y_test))

In [ ]:
# 绘制混淆矩阵
conf_mat = confusion_matrix(y_test, y_pre_rfc)
plt.matshow(conf_mat, cmap=plt.cm.Reds)
for (i, j), z in np.ndenumerate(conf_mat):
    plt.text(j, i, '{0:d}'.format(z), ha='center', va='center')
plt.colorbar()
plt.ylabel('真实结果', fontname="SimSun", fontsize=12)
plt.xlabel('预测结果', fontname="SimSun", fontsize=12)
plt.xticks([0, 1], ["False", "True"], fontsize=10)
plt.yticks([0, 1], ["False", "True"], fontsize=10)
pic_name = r"随机森林分类混淆矩阵"
plt.savefig(r'./pic/{}.png'.format(pic_name), dpi=600, bbox_inches='tight')
plt.show()
plt.close()

print('混淆矩阵：')
print(conf_mat)

print('混淆矩阵：')
print(conf_mat)

# 分类指标文本报告（精确率、召回率、F1值等
print('分类指标报告：')
print(classification_report(y_test, y_pre_rfc))

# 特征重要性
print(rfc.feature_importances_)

In [ ]:
joblib.dump(rfc, "./model/rfc.model")

## XGBoost

In [ ]:
# 贝叶斯优化
xgBoostC = XGBClassifier()

param_grid = {'learning_rate': np.array([0.01, 0.015, 0.025, 0.05, 0.1], dtype=float),
              'gamma': np.array([0, 0.05, 0.1, 0.3, 0.5, 0.7, 0.9, 1], dtype=float),
              "reg_alpha": np.array([0, 0.01, 0.1, 1], dtype=float),
              "reg_lambda": np.array([0, 0.1, 0.5, 1], dtype=float),
              "min_child_weight": np.array([1, 3, 5, 7], dtype=int),
              'subsample': np.array([0.5, 0.6, 0.7, 0.8, 0.9, 1], dtype=float),
              'max_depth': np.arange(start=10, stop=500, step=10, dtype=int),
              'n_estimators': np.arange(start=50, stop=1000, step=50, dtype=int),
              'random_state': [3]}

opt = BayesSearchCV(xgBoostC, param_grid, n_iter=50, n_jobs=3, cv=3)

opt.fit(new_X_train, y_train)

In [ ]:
print(opt.best_params_)
best_params = opt.best_params_

In [ ]:
xgBoostC = XGBClassifier(n_estimators=best_params["n_estimators"],
                         max_depth=best_params["max_depth"],
                         learning_rate=best_params["learning_rate"],
                         gamma=best_params["gamma"],
                         min_child_weight=best_params["min_child_weight"],
                         reg_alpha=best_params["reg_alpha"],
                         reg_lambda=best_params["reg_lambda"],
                         subsample=best_params["subsample"],
                         random_state=best_params["random_state"])
xgBoostC.fit(new_X_train, y_train)

# 预测时间
start = time.time()
print("开始预测！！")
y_pre_xgBoostC = xgBoostC.predict(new_X_test)
end = time.time()
print('预测时间', end - start, 's')

print('正确标签：', y_test)
print('预测结果：', y_pre_xgBoostC)

print('训练集分数：', xgBoostC.score(new_X_train, y_train))
print('测试集分数：', xgBoostC.score(new_X_test, y_test))

In [ ]:
# 绘制混淆矩阵
conf_mat = confusion_matrix(y_test, y_pre_xgBoostC)
plt.matshow(conf_mat, cmap=plt.cm.Blues)
for (i, j), z in np.ndenumerate(conf_mat):
    plt.text(j, i, '{0:d}'.format(z), ha='center', va='center')
plt.colorbar()
plt.ylabel('真实结果', fontname="SimSun", fontsize=12)
plt.xlabel('预测结果', fontname="SimSun", fontsize=12)
plt.xticks([0, 1], ["False", "True"], fontsize=10)
plt.yticks([0, 1], ["False", "True"], fontsize=10)
pic_name = r"XGBoost分类混淆矩阵"
plt.savefig(r'./pic/{}.png'.format(pic_name), dpi=600, bbox_inches='tight')
plt.show()
plt.close()

print('混淆矩阵：')
print(conf_mat)

# 分类指标文本报告（精确率、召回率、F1值等）
print('分类指标报告：')
print(classification_report(y_test, y_pre_xgBoostC))

# 特征重要性
print(xgBoostC.feature_importances_)

In [ ]:
joblib.dump(xgBoostC, "./model/xgBoostC.model")

## 绘制ROC曲线

In [ ]:
blue = (52 / 255, 168 / 255, 255 / 255)
red = (255 / 255, 25 / 255, 65 / 255)
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [ ]:
# 可视化在验证集上的Roc曲线
score_rfc = rfc.predict_proba(new_X_test)[:, 1]
score_xgc = xgBoostC.predict_proba(new_X_test)[:, 1]

#使用roc_curve方法得到三个模型的真正率TP,假正率FP和阈值threshold
fpr_rfc, tpr_rfc, threshold_rfc = roc_curve(y_test, score_rfc)
fpr_xgc, tpr_xgc, threshold_xgc = roc_curve(y_test, score_xgc)

print("随机森林的AUC为:", auc(fpr_rfc, tpr_rfc))
print("XGBoost的AUC为:", auc(fpr_xgc, tpr_xgc))

plt.figure(figsize=(10, 8))
ax = plt.gca()
plt.plot(fpr_rfc, tpr_rfc, linewidth=2, label='随机森林 (AUC={})'.format(str(round(auc(fpr_rfc, tpr_rfc), 4))),
         color=red)
plt.plot(fpr_xgc, tpr_xgc, linewidth=2, label='XGBoost (AUC={})'.format(str(round(auc(fpr_xgc, tpr_xgc), 4))),
         color=blue)
plt.rcParams['font.sans-serif'] = ['Times New Roman']
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.rcParams['axes.unicode_minus'] = False
ax.plot([0, 1], [0, 1], 'k--')
plt.xlabel("假正率(FP)", fontname='SimSun', fontsize=20)
plt.ylabel("真正率(TP)", fontname='SimSun', fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.legend(loc="best", fancybox=True, fontsize=15, shadow=True)
pic_name = r"ROC曲线"
plt.savefig(r'./pic/{}.png'.format(pic_name), dpi=600, bbox_inches='tight')
plt.show()
plt.close()

# 回归问题

In [ ]:
x_label = ["period", "site_no", "wg", "xig", "stru_damping", "gamma_t"]
y_label = ["miu", "zeta", "kappa"]
df_ok_origin = df[df.ok==True]
# df_ok_origin = df
df_ok = df_ok_origin.copy()

In [ ]:
X = df_ok.loc[:, x_label].values
y = df_ok.loc[:, y_label].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=2, shuffle=True)

In [ ]:
y

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.view_init(elev=10, azim=0)
ax.scatter(y[:,0], y[:,1], y[:,2], alpha=0.3)
ax.set_xlabel(r'$\mu$', fontsize=15)
ax.set_ylabel(r'$\xi$', fontsize=15)
ax.set_zlabel(r'$\kappa$', fontsize=15)
pic_name = r"设计参数分布视角1"
plt.savefig(r'./pic/{}.png'.format(pic_name), dpi=600)
plt.show()
plt.close()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.view_init(elev=10, azim=160)
ax.scatter(y[:,0], y[:,1], y[:,2], alpha=0.2)
ax.set_xlabel(r'$\mu$', fontsize=15)
ax.set_ylabel(r'$\xi$', fontsize=15)
ax.set_zlabel(r'$\kappa$', fontsize=15)
fig.subplots_adjust()
pic_name = r"设计参数分布视角2"
plt.savefig(r'./pic/{}.png'.format(pic_name), dpi=600)
plt.show()
plt.close()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.view_init(elev=10, azim=230)
ax.scatter(y[:,0], y[:,1], y[:,2], alpha=0.2)
ax.set_xlabel(r'$\mu$', fontsize=15)
ax.set_ylabel(r'$\xi$', fontsize=15)
ax.set_zlabel(r'$\kappa$', fontsize=15)
pic_name = r"设计参数分布视角3"
plt.savefig(r'./pic/{}.png'.format(pic_name), dpi=600)
plt.show()
plt.close()

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
y_train_mu = y_train[:, 0]
y_test_mu = y_test[:, 0]
y_mu = y[:, 0]
y_train_zeta = y_train[:, 1]
y_test_zeta = y_test[:, 1]
y_zeta = y[:, 1]
y_train_kappa = y_train[:, 2]
y_test_kappa = y_test[:, 2]
y_kappa = y[:, 2]

In [ ]:
new_X_train = X_train
new_X_test = X_test

In [ ]:
# 通用函数

def plot_regression_results(ax, y_true, y_pred, title, scores, type, label):
    """预测目标与真实目标的散点图。"""
    blue = (60 / 255, 160 / 255, 255 / 255)
    red = (255 / 255, 25 / 255, 65 / 255)
    if type:
        color1 = red
        color2 = (0 / 255, 0 / 255, 255 / 255)
    else:
        color1 = blue
        color2 = (255 / 255, 0 / 255, 0 / 255)
    ax.plot([y_true.min(), y_true.max()],
            [y_true.min(), y_true.max()], '--', linewidth=2, color=color2)
    ax.scatter(y_true, y_pred, alpha=0.25, color=color1)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.get_xaxis().tick_bottom()
    ax.get_yaxis().tick_left()
    ax.spines['left'].set_position(('outward', 10))
    ax.spines['bottom'].set_position(('outward', 10))
    xlim_dis = (y_true.max() - y_true.min()) * 0.08
    ylim_dis = (y_pred.max() - y_pred.min()) * 0.08
    ax.set_xlim([y_true.min() - xlim_dis * 0.5, y_true.max() + xlim_dis])
    ax.set_ylim([y_pred.min() - ylim_dis * 0.5, y_pred.max() + ylim_dis])
    ax.set_xlabel('真实值,' + label, fontname='SimSun', fontsize=12)
    ax.set_ylabel('预测值', fontname='SimSun', fontsize=12)
    extra = plt.Rectangle((0, 0), 0, 0, fc="w", fill=False,
                          edgecolor='none', linewidth=0)
    ax.legend([extra], [scores], loc='upper left', shadow=True)
    ax.set_title(title, fontname='SimSun', fontsize=15)

def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))


def plot_true_predict_pic(y_true, y_pred, model_name, type, label):
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    msle = mean_squared_log_error(y_true, y_pred)
    rmsle = np.sqrt(msle)

    plt.figure(figsize=(6, 5))
    ax = plt.gca()
    plot_regression_results(
        ax, y_true, y_pred,
        model_name,
        (r'$R^2={:.4f}$' + '\n' +
         r'$MAE={:.4f}$' + '\n' +
         r'$RMSE={:.4f}$' + '\n' +
         r'$MAPE={:.4f}$' + '\n' +
         r'$RMSLE={:.4f}$')
        .format(r2, mae, rmse, mape, rmsle), type, label)


def plot_true_predict_pic_time(y_true, y_pred, model_name, type, label, pred_time):
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    # mape = 0
    msle = mean_squared_log_error(y_true, y_pred)
    # msle = 0
    rmsle = np.sqrt(msle)

    plt.figure(figsize=(6, 5))
    ax = plt.gca()
    plot_regression_results(
        ax, y_true, y_pred,
        model_name,
        (r'$R^2={:.4f}$' + '\n' +
         r'$MAE={:.4f}$' + '\n' +
         r'$RMSE={:.4f}$' + '\n' +
         r'$MAPE={:.4f}$' + '\n' +
         r'$RMSLE={:.4f}$' + '\n' +
         r'预测时间$={:.4f}s$')
        .format(r2, mae, rmse, mape, rmsle, pred_time), type, label)

def plot_true_predict_pic_time2(y_true, y_pred, model_name, type, label, pred_time):
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)

    plt.figure(figsize=(6, 5))
    ax = plt.gca()
    plot_regression_results(
        ax, y_true, y_pred,
        model_name,
        (r'$R^2={:.4f}$' + '\n' +
         r'$MAE={:.4f}$' + '\n' +
         r'$RMSE={:.4f}$' + '\n' +
         r'预测时间$={:.4f}s$')
        .format(r2, mae, rmse, pred_time), type, label)

## 随机森林

In [ ]:
rfr_mu = RandomForestRegressor()
rfr_zeta = RandomForestRegressor()
rfr_kappa = RandomForestRegressor()

param_grid = {'max_depth': np.arange(start=10, stop=500, step=10, dtype=int),
              'n_estimators': np.arange(start=50, stop=1000, step=50, dtype=int),
              'max_features': np.arange(start=1, stop=12, step=1, dtype=int),
              'min_samples_leaf': np.array([2, 5, 10, 15], dtype=int),
              'min_samples_split': np.array([2, 5, 10, 15], dtype=int),
              'random_state': [3]}
opt_mu = BayesSearchCV(rfr_mu, param_grid, n_iter=50, n_jobs=5, cv=3)
opt_zeta = BayesSearchCV(rfr_zeta, param_grid, n_iter=50, n_jobs=5, cv=3)
opt_kappa = BayesSearchCV(rfr_kappa, param_grid, n_iter=50, n_jobs=5, cv=3)

opt_mu.fit(new_X_train, y_train_mu)
opt_zeta.fit(new_X_train, y_train_zeta)
opt_kappa.fit(new_X_train, y_train_kappa)

In [ ]:
best_params_mu = opt_mu.best_params_
best_params_mu

In [ ]:
best_params_zeta = opt_zeta.best_params_
best_params_zeta

In [ ]:
best_params_kappa = opt_kappa.best_params_
best_params_kappa

### mu

In [ ]:
rfr_mu = RandomForestRegressor(max_depth=best_params_mu["max_depth"],
                                      n_estimators=best_params_mu["n_estimators"],
                                      max_features=best_params_mu["max_features"],
                                      min_samples_leaf=best_params_mu["min_samples_leaf"],
                                      min_samples_split=best_params_mu["min_samples_split"],
                                      random_state=best_params_mu["random_state"]
                                      )
rfr_mu.fit(new_X_train, y_train_mu)
# rfr_mu.fit(X, y_mu)
# 预测时间
start = time.time()
print("开始预测！！")
train_pred_mu = rfr_mu.predict(new_X_train)
pred_mu = rfr_mu.predict(new_X_test)
end = time.time()
print('预测时间', end - start, 's')
print(rmsle(y_train_mu, train_pred_mu))
plot_true_predict_pic_time(y_test_mu, pred_mu, "随机森林模型", True, r'$\mu$', end-start)
pic_name = r"随机森林_mu"
plt.savefig(r'./pic/{}.png'.format(pic_name), dpi=600, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
plot_true_predict_pic_time(y_train_mu, train_pred_mu, "随机森林模型", True, r'$\mu$', end-start)

### zeta

In [ ]:
rfr_zeta = RandomForestRegressor(max_depth=best_params_zeta["max_depth"],
                                      n_estimators=best_params_zeta["n_estimators"],
                                      max_features=best_params_zeta["max_features"],
                                      min_samples_leaf=best_params_zeta["min_samples_leaf"],
                                      min_samples_split=best_params_zeta["min_samples_split"],
                                      random_state=best_params_zeta["random_state"]
                                      )
rfr_zeta.fit(new_X_train, y_train_zeta)
# rfr_zeta.fit(X, y_zeta)
# 预测时间
start = time.time()
print("开始预测！！")
train_pred_zeta = rfr_zeta.predict(new_X_train)
pred_zeta = rfr_zeta.predict(new_X_test)
print(rmsle(y_train_zeta, train_pred_zeta))
end = time.time()
print('预测时间', end - start, 's')
plot_true_predict_pic_time(y_test_zeta, pred_zeta, "随机森林模型", True, r'$\xi$', end-start)
pic_name = r"随机森林_zeta"
plt.savefig(r'./pic/{}.png'.format(pic_name), dpi=600, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
np.min(pred_zeta)

In [ ]:
plot_true_predict_pic_time(y_train_zeta, train_pred_zeta, "随机森林模型", True, r'$\mu$', end-start)

### kappa

In [ ]:
rfr_kappa = RandomForestRegressor(max_depth=best_params_kappa["max_depth"],
                                  n_estimators=best_params_kappa["n_estimators"],
                                  max_features=best_params_kappa["max_features"],
                                  min_samples_leaf=best_params_kappa["min_samples_leaf"],
                                  min_samples_split=best_params_kappa["min_samples_split"],
                                  random_state=best_params_kappa["random_state"]
                                  )
rfr_kappa.fit(new_X_train, y_train_kappa)
# rfr_kappa.fit(X, y_kappa)
# 预测时间
start = time.time()
print("开始预测！！")
train_pred_kappa = rfr_kappa.predict(new_X_train)
pred_kappa = rfr_kappa.predict(new_X_test)
print(rmsle(y_train_kappa, train_pred_kappa))
end = time.time()
print('预测时间', end - start, 's')
plot_true_predict_pic_time(y_test_kappa, pred_kappa, "随机森林模型", True, r'$\kappa$', end-start)
pic_name = r"随机森林_kappa"
plt.savefig(r'./pic/{}.png'.format(pic_name), dpi=600, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
plot_true_predict_pic_time(y_train_kappa, train_pred_kappa, "随机森林模型", True, r'$\mu$', end-start)

In [ ]:
joblib.dump(rfr_mu, "./model/rfr_mu.model")
joblib.dump(rfr_zeta, "./model/rfr_zeta.model")
joblib.dump(rfr_kappa, "./model/rfr_kappa.model")

## XGBoost

In [ ]:
df.describe()

In [ ]:
x_label = ["period", "site_no", "wg", "xig", "stru_damping", "gamma_t"]
y_label = ["miu", "zeta", "kappa"]
df_ok_origin = df[df.ok == True]
# df_ok_origin = df
df_ok = df_ok_origin.copy()
X = df_ok.loc[:, x_label].values
y = df_ok.loc[:, y_label].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=2, shuffle=True)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

from sklearn.preprocessing import StandardScaler

y_train_mu = y_train[:, 0].reshape(-1, 1)
y_test_mu = y_test[:, 0].reshape(-1, 1)
scaler_mu = StandardScaler()
y_train_mu_sc = scaler_mu.fit_transform(y_train_mu)
y_test_mu_sc = scaler_mu.transform(y_test_mu)


y_train_zeta = y_train[:, 1].reshape(-1, 1)
y_test_zeta = y_test[:, 1].reshape(-1, 1)
scaler_zeta = StandardScaler()
y_train_zeta_sc = scaler_zeta.fit_transform(y_train_zeta)
y_test_zeta_sc = scaler_zeta.transform(y_test_zeta)



y_train_kappa = y_train[:, 2].reshape(-1, 1)
y_test_kappa = y_test[:, 2].reshape(-1, 1)
scaler_kappa = StandardScaler()
y_train_kappa_sc = scaler_kappa.fit_transform(y_train_kappa)
y_test_kappa_sc = scaler_kappa.transform(y_test_kappa)

In [ ]:
param_grid = {'learning_rate': np.array([0.01, 0.015, 0.025, 0.05, 0.1], dtype=float),
               'gamma': np.array([0, 0.05, 0.1, 0.3, 0.5, 0.7, 0.9, 1], dtype=float),
               "reg_alpha": np.array([0, 0.01, 0.1, 1], dtype=float),
               "reg_lambda": np.array([0, 0.1, 0.5, 1], dtype=float),
               "min_child_weight": np.array([1, 3, 5, 7], dtype=int),
               'subsample': np.array([0.5, 0.6, 0.7, 0.8, 0.9, 1], dtype=float),
               'max_depth': np.arange(start=10, stop=500, step=10, dtype=int),
               'n_estimators': np.arange(start=50, stop=1000, step=50, dtype=int),
               'random_state': [3]}
xgb_mu = XGBRegressor()
xgb_zeta = XGBRegressor()
xgb_kappa = XGBRegressor()

xgb_opt_mu = BayesSearchCV(xgb_mu, param_grid, n_iter=5, n_jobs=5, cv=3)
xgb_opt_zeta = BayesSearchCV(xgb_zeta, param_grid, n_iter=5, n_jobs=5, cv=3)
xgb_opt_kappa = BayesSearchCV(xgb_kappa, param_grid, n_iter=5, n_jobs=5, cv=3)

xgb_opt_mu.fit(new_X_train, y_train_mu_sc)
xgb_opt_zeta.fit(new_X_train, y_train_zeta_sc)
xgb_opt_kappa.fit(new_X_train, y_train_kappa_sc)

# xgb_opt_mu.fit(X, y_mu)
# xgb_opt_zeta.fit(X, y_zeta)
# xgb_opt_kappa.fit(X, y_kappa)

In [ ]:
xgb_best_params_mu = xgb_opt_mu.best_params_
xgb_best_params_mu

In [ ]:
xgb_best_params_zeta = xgb_opt_zeta.best_params_
xgb_best_params_zeta

In [ ]:
xgb_best_params_kappa = xgb_opt_kappa.best_params_
xgb_best_params_kappa

### mu

In [ ]:
xgb_mu = XGBRegressor(learning_rate=xgb_best_params_mu["learning_rate"],
                             reg_alpha=xgb_best_params_mu["reg_alpha"],
                             reg_lambda=xgb_best_params_mu["reg_lambda"],
                             min_child_weight=xgb_best_params_mu["min_child_weight"],
                             subsample=xgb_best_params_mu["subsample"],
                             max_depth=xgb_best_params_mu["max_depth"],
                             n_estimators=xgb_best_params_mu["n_estimators"],
                             random_state=xgb_best_params_mu["random_state"])
xgb_mu.fit(new_X_train, y_train_mu_sc)
# 预测时间
start = time.time()
print("开始预测！！")
xgb_train_pred_mu = xgb_mu.predict(new_X_train)
xgb_train_pred_mu = scaler_mu.inverse_transform(xgb_train_pred_mu.reshape(-1, 1))
xgb_pred_mu = xgb_mu.predict(new_X_test)
xgb_pred_mu = scaler_mu.inverse_transform(xgb_pred_mu.reshape(-1, 1))
print(rmsle(y_train_mu, xgb_train_pred_mu))
end = time.time()
print('预测时间', end - start, 's')
plot_true_predict_pic_time2(y_test_mu, xgb_pred_mu, "XGBoost模型", False, r'$\mu$', end-start)
pic_name = r"XGBoost_mu"
plt.savefig(r'./pic/{}.png'.format(pic_name), dpi=600, bbox_inches='tight')
plt.show()
plt.close()

### zeta

In [ ]:
xgb_zeta = XGBRegressor(learning_rate=xgb_best_params_zeta["learning_rate"],
                             reg_alpha=xgb_best_params_zeta["reg_alpha"],
                             reg_lambda=xgb_best_params_zeta["reg_lambda"],
                             min_child_weight=xgb_best_params_zeta["min_child_weight"],
                             subsample=xgb_best_params_zeta["subsample"],
                             max_depth=xgb_best_params_zeta["max_depth"],
                             n_estimators=xgb_best_params_zeta["n_estimators"],
                             random_state=xgb_best_params_zeta["random_state"])
xgb_zeta.fit(new_X_train, y_train_zeta_sc)
# xgb_zeta.fit(X, y_zeta)
# 预测时间
start = time.time()
print("开始预测！！")
xgb_train_pred_zeta = xgb_zeta.predict(new_X_train)
xgb_train_pred_zeta = scaler_zeta.inverse_transform(xgb_train_pred_zeta.reshape(-1, 1))
xgb_pred_zeta = xgb_zeta.predict(new_X_test)
xgb_pred_zeta = scaler_zeta.inverse_transform(xgb_pred_zeta.reshape(-1, 1))
print(rmsle(y_train_zeta, xgb_train_pred_zeta))
end = time.time()
print('预测时间', end - start, 's')
plot_true_predict_pic_time2(y_test_zeta, xgb_pred_zeta, "XGBoost模型", False, r'$\zeta$', end-start)
pic_name = r"XGBoost_zeta"
plt.savefig(r'./pic/{}.png'.format(pic_name), dpi=600, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
np.min(xgb_pred_zeta)

### kappa

In [ ]:
xgb_kappa = XGBRegressor(learning_rate=xgb_best_params_kappa["learning_rate"],
                             reg_alpha=xgb_best_params_kappa["reg_alpha"],
                             reg_lambda=xgb_best_params_kappa["reg_lambda"],
                             min_child_weight=xgb_best_params_kappa["min_child_weight"],
                             subsample=xgb_best_params_kappa["subsample"],
                             max_depth=xgb_best_params_kappa["max_depth"],
                             n_estimators=xgb_best_params_kappa["n_estimators"],
                             random_state=xgb_best_params_kappa["random_state"])
xgb_kappa.fit(new_X_train, y_train_kappa_sc)
# xgb_kappa.fit(X, y_kappa)
# 预测时间
start = time.time()
print("开始预测！！")
xgb_train_pred_kappa = xgb_kappa.predict(new_X_train)
xgb_train_pred_kappa = scaler_kappa.inverse_transform(xgb_train_pred_kappa.reshape(-1, 1))
xgb_pred_kappa = xgb_kappa.predict(new_X_test)
xgb_pred_kappa = scaler_kappa.inverse_transform(xgb_pred_kappa.reshape(-1, 1))
print(rmsle(y_train_kappa, xgb_train_pred_kappa))
end = time.time()
print('预测时间', end - start, 's')
plot_true_predict_pic_time(y_test_kappa, xgb_pred_kappa, "XGBoost模型", False, r'$\kappa$', end-start)
pic_name = r"XGBoost_kappa"
plt.savefig(r'./pic/{}.png'.format(pic_name), dpi=600, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
joblib.dump(xgb_mu, "./model/xgb_mu.model")
joblib.dump(xgb_kappa, "./model/xgb_kappa.model")
joblib.dump(xgb_zeta, "./model/xgb_zeta.model")